In [45]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler , RobustScaler
from scipy.stats import boxcox,skew
from sklearn.linear_model import LogisticRegression ,LogisticRegressionCV
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
warnings.filterwarnings("ignore")
df = pd.read_csv("/Users/hajiaga/Downloads/creditcard.csv")


In [ ]:
#Code which is fully written me and mentor

def get_outliers(data , column):
    Q1 = data[column].quantile(0.25)
    Q3 = data[column].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - IQR*1.5
    upper_bound = Q3 + IQR*1.5
    return [data.loc[(data[column].astype(float) < lower_bound) | (data[column].astype(float) > upper_bound)].shape[0] , lower_bound ,upper_bound]

lowers , uppers = [] , []
for col in df.columns:
    lowers.append(get_outliers(df,col)[1])
    uppers.append(get_outliers(df,col)[2])
print(lowers)
print(uppers)
np.sum((df>lowers) & ( df<uppers),axis=1)>8
df = df[np.sum((df>lowers) & ( df<uppers),axis=1)>23]
df.info()
df["Class"]

In [47]:
def replace_outliers_with_bounds(data):
    for column in data.columns:
        Q1 = data[column].quantile(0.25)
        Q3 = data[column].quantile(0.75)
        IQR = Q3 - Q1
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        if column != "Class":
            data[column] = np.where(data[column] < lower_bound, lower_bound, data[column])
            data[column] = np.where(data[column] > upper_bound, upper_bound, data[column])
    return data
df = replace_outliers_with_bounds(df)

In [48]:
# for col in df.columns:
#     fig , (bfr , aft) = plt.subplots(nrows=1,ncols=2,figsize=(15,3))
#     sns.histplot(df[col] , ax=bfr)
#     sns.histplot(df[col].apply(np.log1p) , ax=aft)
#     bfr.set(title= col)
#     aft.set(title="Buda sonraki")
#     plt.show()


In [ ]:
num_cols = df.dtypes[df.dtypes != "object"].index
skewed_feats = df[num_cols].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})

skewed_feats =skewness[skewness>0.70].index
for i in skewed_feats:
    if i != "Class":
        print(np.min(df[i]),i)
        df[i] +=5
        fig , (bfr,aft) = plt.subplots(ncols=2,nrows=1,figsize = (15,3))
        sns.histplot(df[i],ax=bfr)
        df[i] , lam = boxcox(df[i])
        sns.histplot(df[i],ax=aft)
        plt.show()
        bfr.set(title= i)
        aft.set(title="Buda sonraki")

df.select_dtypes(include="O").columns
print(skewed_feats)

In [50]:
#model qurmaq
#Bilmediyim bir error gelir duzelde bilmedim ne qeder arasdirsamda
X_tr , X_te , Y_tr , Y_te  = train_test_split(df.drop('Class', axis=1), df['Class'], test_size=0.2, random_state=42, shuffle=True)
pipeline = make_pipeline(StandardScaler(),LogisticRegression(random_state=42))

pipeline.fit(X_tr,Y_tr)

print(r2_score(Y_tr,pipeline.predict(X_tr)))
print(r2_score(Y_te,pipeline.predict(X_te)))


0.041352159690537516
0.10495320475235947
